In [154]:

#Converting categorical data into numbers with Pandas and Scikit-learn
#feature extraction. 
#When it involves a lot of manual work, this is often referred to as feature engineering.

In [69]:
import numpy
import sklearn
from sklearn.feature_extraction.text import CountVectorizer

In [70]:
content= ["hello How ARE YOU there", "How is  ARE everything everyting","I am not sure how this is possible", "How are you"]

In [71]:
vectorizer = CountVectorizer(min_df=1)
X_train = vectorizer.fit_transform(content)s

In [72]:
vectorizer.get_feature_names()

[u'am',
 u'are',
 u'everything',
 u'everyting',
 u'hello',
 u'how',
 u'is',
 u'not',
 u'possible',
 u'sure',
 u'there',
 u'this',
 u'you']

In [73]:
print X_train.shape
print X_train.toarray()ssd

(4, 13)
[[0 1 0 0 1 1 0 0 0 0 1 0 1]
 [0 1 1 1 0 1 1 0 0 0 0 0 0]
 [1 0 0 0 0 1 1 1 1 1 0 1 0]
 [0 1 0 0 0 1 0 0 0 0 0 0 1]]


In [74]:
new_post = "How ARE how YOU there"
new_post_vec = vectorizer.transform([new_post])
print new_post_vec

  (0, 1)	1
  (0, 5)	2
  (0, 10)	1
  (0, 12)	1


In [161]:
print new_post_vec.toarray()

[[0 1 0 0 0 1 0 0 0 0 1 0 1]]


In [162]:
#Similarity Calculations; Calculate Eculidean Distance between the count vectors of the new post and ll the old posts as below:

In [163]:
import scipy as sp
def dist_raw(v1,v2):
    delta= v1-v2
    return sp.linalg.norm(delta.toarray()) #norm() calculates the Eculidean norm i.e. shortest distance

def dist_norm(v1,v2):
    v1_normalized = v1/sp.linalg.norm(v1.toarray())
    v2_normalized = v2/sp.linalg.norm(v2.toarray())
    delta= v1_normalized-v2_normalized
    return sp.linalg.norm(delta.toarray()) #norm() calculates the Eculidean norm i.e. shortest distance

In [76]:
import sys
best_doc = None
best_i = None
num_samples = len(content)

def best_match(X_train,new_post_vec):
    best_dist = sys.maxint
    for i in range(0, num_samples):
        post = content[i]
        if post == new_post:
            continue
        post_vec = X_train.getrow(i)
        #d = dist_raw(post_vec, new_post_vec)
        d = dist_norm(post_vec, new_post_vec)
        print "===Post %i with dist = %.2f: %s"%(i,d,post)
        if d< best_dist:
            best_dist = d
            best_i = i
    print "Best post is %i with dist = %.4f"%(best_i,best_dist)
best_match(X_train, new_post_vec)

===Post 0 with dist = 0.56: hello How ARE YOU there
===Post 1 with dist = 0.99: How is  ARE everything everyting
===Post 2 with dist = 1.20: I am not sure how this is possible
===Post 3 with dist = 0.50: How are you
Best post is 3 with dist = 0.5042


In [77]:
print X_train.getrow(0).toarray()
print new_post_vec.toarray()

[[0 1 0 0 1 1 0 0 0 0 1 0 1]]
[[0 1 0 0 0 2 0 0 0 0 1 0 1]]


In [78]:
#Removing less important words
#Remove more frequent words that do not help to distinguish netween different texts. 
#MODIFY YOUR Vectorizer
vectorizer2 = CountVectorizer(min_df =1, stop_words='english')
sorted(vectorizer2.get_stop_words())[0:10]

['a',
 'about',
 'above',
 'across',
 'after',
 'afterwards',
 'again',
 'against',
 'all',
 'almost']

In [79]:
X_train2 = vectorizer2.fit_transform(content)
vectorizer2.get_feature_names()
new_post_vec2 = vectorizer2.transform([new_post])
best_match(X_train2, new_post_vec2)

===Post 0 with dist = 1.00: hello How ARE YOU there
===Post 1 with dist = 1.00: How is  ARE everything everyting
===Post 2 with dist = 1.00: I am not sure how this is possible
===Post 3 with dist = 0.00: How are you
Best post is 3 with dist = 0.0000


In [80]:
# Use NLTK to reduce words to their stem i.e. origin
import nltk.stem
s= nltk.stem.SnowballStemmer('english')
s.stem("graphics")

u'graphic'

In [83]:
'''Use StemmedCountVectorizer to do:
1. lower casing the raw post in the preprossing step done in parent calss.
2. Extracting all individual words in the tokenization step in parent class.
3. Converting each word into its stemmed version.'''
english_stemmer = nltk.stem.SnowballStemmer('english')
class StemmedCountVectorizer(CountVectorizer):
    english_stemmer = nltk.stem.SnowballStemmer('english')
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer,self).build_analyzer()
        return lambda doc: (english_stemmer.stem(w) for w in analyzer(doc))

In [92]:
stem_vectorizer = StemmedCountVectorizer(min_df =1, stop_words='english')
X_train3 = stem_vectorizer.fit_transform(content)
print stem_vectorizer.get_feature_names()
print X_train3.toarray()

new_post_vec3 = stem_vectorizer.transform([new_post])
best_match(X_train3, new_post_vec3)
print("new post:"),new_post

[u'everyt', u'hello', u'possibl', u'sure']
[[0 1 0 0]
 [1 0 0 0]
 [0 0 1 1]
 [0 0 0 0]]
===Post 0 with dist = 1.00: hello How ARE YOU there
===Post 1 with dist = 1.00: How is  ARE everything everyting
===Post 2 with dist = 1.00: I am not sure how this is possible
===Post 3 with dist = 0.00: How are you
Best post is 3 with dist = 0.0000
new post: How ARE how YOU there


FootNotes:
    
    What does a rater sees when he rates an android app? == Extrinsic Features
    What an android app inherits that influences app rating? == Intrinsic Features
    
    
    Vectors to predict: 1. 5-star count, 4-star count, 3-star-count, 2-star count, 1-star count.
    Because, average app-rating depends upon the values of these values. Also on current rating of the app.
    

In [2]:
#Read CSV
import pandas as pd
from pandas import *
from numpy import *
import numpy as np
import os
from pandas import DataFrame
import numpy
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
import scipy as sp
# Use NLTK to reduce words to their stem i.e. origin
import nltk.stem
# Use NLTK to reduce words to their stem i.e. origin
import nltk.stem

In [3]:
app_file = '../data/big-data-csv.csv'
appdf = pd.read_csv(app_file,sep=',')
appdf.head(2)

,Unnamed: 0,Category,Score,Description,Price,PublicationDate,AppSize,Name,ContentRating,LastUpdateDate,Instalations,IsTopDeveloper,HaveInAppPurchases,IsFree,Developer
0,0,NEWS_AND_MAGAZINES,5.000000,Read the most popular newspapers from Sweden ...,0,2015-07-08T03:00:00.000Z,2.9,Sweden News,Everyone 10+,2015-07-08T03:00:00.000Z,50 - 100,False,False,True,News Now
1,1,MEDIA_AND_VIDEO,2.882353,Sweden Tv channels guide. Tv Sweden include lo...,0,2015-07-25T03:00:00.000Z,2.8,Tv Sweden,Everyone,2015-07-25T03:00:00.000Z,"5,000 - 10,000",False,False,True,QSC


In [7]:
col_cat = appdf.Category
col_cat.head(2)d

0    NEWS_AND_MAGAZINES
1       MEDIA_AND_VIDEO
Name: Category, dtype: object

In [4]:
appdf.columns.values.tolist()

['Unnamed: 0',
 'Category',
 'Score',
 'Description',
 'Price',
 'PublicationDate',
 'AppSize',
 'Name',
 'ContentRating',
 'LastUpdateDate',
 'Instalations',
 'IsTopDeveloper',
 'HaveInAppPurchases',
 'IsFree',
 'Developer']

In [8]:
len(col_cat.unique())
col_cat.unique()

array(['NEWS_AND_MAGAZINES', 'MEDIA_AND_VIDEO', 'ENTERTAINMENT', 'FINANCE',
       'MUSIC_AND_AUDIO', 'TRAVEL_AND_LOCAL', 'EDUCATION', 'BUSINESS',
       'PERSONALIZATION', 'TRANSPORTATION', 'SPORTS', 'SOCIAL',
       'COMMUNICATION', 'PHOTOGRAPHY', 'LIFESTYLE', 'HEALTH_AND_FITNESS',
       'TOOLS', 'PRODUCTIVITY', 'WEATHER', 'BOOKS_AND_REFERENCE',
       'GAME_TRIVIA', 'MEDICAL', 'GAME_PUZZLE', 'GAME_CASUAL', 'SHOPPING',
       'GAME_MUSIC', 'GAME_ACTION', 'GAME_ARCADE', 'GAME_SIMULATION',
       'GAME_CARD', 'GAME_CASINO', 'LIBRARIES_AND_DEMO',
       'GAME_EDUCATIONAL', 'GAME_SPORTS', 'GAME_WORD', 'GAME_RACING',
       'GAME_ROLE_PLAYING', 'GAME_BOARD', 'COMICS', 'GAME_STRATEGY',
       'GAME_ADVENTURE'], dtype=object)

In [4]:
'''Use StemmedCountVectorizer to do:
1. lower casing the raw post in the preprossing step done in parent calss.
2. Extracting all individual words in the tokenization step in parent class.
3. Converting each word into its stemmed version.'''
import nltk.stem

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        english_stemmer = nltk.stem.SnowballStemmer('english')
        analyzer = super(StemmedCountVectorizer,self).build_analyzer()
        return lambda doc: (english_stemmer.stem(w) for w in analyzer(doc))

In [21]:
'''Returns a vectorized ND dataframe, vectorized ndarray, and an instance of the vectorizer Class used to transform.'''
def vectorize_column(dataframe,column_name,vectorizer=None):
    if vectorizer is None:
        print("No Vectorizer is explicitly specified. Using CountVectorizer as default one. ")
        column_vectorizer = CountVectorizer(min_df=1)
    else:
        column_vectorizer = vectorizer
    
    if column_name in dataframe.columns.values.tolist():
        column_df = dataframe[column_name] #select all the samples from the column passed as param.
        fmatrix = column_vectorizer.fit_transform(column_df) #convert text features to numerical vectors
        #print column_vectorizer.get_feature_names() #print the features names formed after fitting a categorical data
        #print("vectorized into matrix of shape"), fmatrix.toarray().shape
        dataframe_f = pd.DataFrame(fmatrix.toarray(), columns=column_vectorizer.get_feature_names())
        print("formed dataframe of size:("),dataframe_f.index.max()+1,",", dataframe_f.head(1).shape[1],")"
        
        return dataframe_f, fmatrix, column_vectorizer
    else:
        print("No column found")
        
'''Returns a vectorized ND dataframe, vectorized ndarray, and an instance of the vectorizer Class used to transform.'''
def vectorize_column2(dataframe,column_name,vectorizer=None, n_samples=None):
    if vectorizer is None:
        print("No Vectorizer is explicitly specified. Using CountVectorizer as default one. ")
        column_vectorizer = CountVectorizer(min_df=1)
    else:
        column_vectorizer = vectorizer
    
    if column_name in dataframe.columns.values.tolist():
        
        if n_samples is None:
            column_df = dataframe[column_name] #select all the samples from the column passed as param. 
            print len(column_df)
        else:
            column_df = dataframe[column_name].iloc[0:n_samples] #select all the samples from the column passed as param.
            print len(column_df)

        fmatrix = column_vectorizer.fit_transform(column_df) #convert text features to numerical vectors
        #print column_vectorizer.get_feature_names() #print the features names formed after fitting a categorical data
        #print("vectorized into matrix of shape"), fmatrix.toarray().shape
        dataframe_f = pd.DataFrame(fmatrix.toarray(), columns=column_vectorizer.get_feature_names())
        print("formed dataframe of size:("),dataframe_f.index.max()+1,",", dataframe_f.head(1).shape[1],")"
        
        return dataframe_f, fmatrix, column_vectorizer
    else:
        print("No column found")


In [7]:
'''Possible type of count vectorizer that could be used.
    Examples: 
    column_vectorizer = CountVectorizer(min_df=1)
    column_vectorizer = CountVectorizer(min_df =1, stop_words='english') 
    print column_vectorizer.get_feature_names()
'''

stem_vectorizer = StemmedCountVectorizer(min_df =1, stop_words='english')
newfeature, fmatrix, column_vectorizer = vectorize_column(appdf, 'Category', stem_vectorizer)
newfeature.head(5)

formed dataframe of size:( 100000 , 41 )


,books_and_refer,busi,comic,communic,educ,entertain,financ,game_act,game_adventur,game_arcad,...,person,photographi,product,shop,social,sport,tool,transport,travel_and_loc,weather
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [89]:
'''Computes eculidean distance between two normalized vectors v1 and v2'''
def dist_norm(v1,v2):
    v1_normalized = v1/sp.linalg.norm(v1.toarray())
    v2_normalized = v2/sp.linalg.norm(v2.toarray())
    delta= v1_normalized - v2_normalized
    return sp.linalg.norm(delta.toarray()) #norm() calculates the Eculidean norm i.e. shortest distance"

#### '''Analysis of 'Category' data-columns
#Each of the application has only one 'category' so the each of the category is equi-distance from all other.
Though similarity of each of the values of category is same, the category-name itself might not effect the rating equally.
That is why they are inluded as training features.'''

In [94]:
import sys
best_doc = None
best_i = None

def best_match(column_vectorizer, fmatrix,text_to_compare):
    n_samples = 100 # fmatrix.shape[0]
    best_dist = sys.maxint
    vect_to_compare = column_vectorizer.transform(text_to_compare)
    for i in range(0, n_samples):
        text_in_column = col_cat[i]
        if text_in_column == text_to_compare[0]:
            continue
        vector_for_column_text = fmatrix.getrow(i)
        #d = dist_raw(post_vec, new_post_vec)
        d = dist_norm(vector_for_column_text, vect_to_compare)
        print "===Category of app- %i with dist = %.2f: %s"%(i,d,text_in_column)
        if d < best_dist:
            best_dist = d
            best_i = i
    print "Best text in category is %i with dist = %.4f"%(best_i,best_dist)
print type([col_cat[4]])
#best_match(column_vectorizer,fmatrix, [col_cat[4]])

100000
<type 'list'>


#### #Analysis of Description Field


In [109]:
def stat_vectorized_matrix(vectorized_array,vectorizer_type=None):
   
    #count the number of features generated,
    print type(vectorized_array)
    m, n = vectorized_array.shape
    count_non_zero_cells = np.count_nonzero(vectorized_array)
    print("vectorizer_type:"),type(column_vectorizer)
    print("number of features generated: "), n
    print("n_samples:"), m
    #count the number of non-zero entries,
    print("% of non-zero values:"), count_non_zero_cells/float(m*n) * 100


In [110]:
col_desc = appdf.Description

df_desc = pd.DataFrame(col_desc, columns=['Description']).iloc[280:285]
print df_desc.to_string()
df_desc.to_csv("test_file.csv",sep=",")

stem_vectorizer = StemmedCountVectorizer(min_df =1, stop_words='english')
newfeature, fmatrix, column_vectorizer = vectorize_column2(df_desc, 'Description', stem_vectorizer, n_samples=100)
print("feature_names"), column_vectorizer.get_feature_names()
stat_vectorized_matrix(fmatrix.toarray(), column_vectorizer)
newfeature.head(10)

                                           Description
280  臺灣北中南東怎麼玩才最深入？旅遊行程該怎麼安排才最盡興？您不可不知的巷內美食、私房景點以及特...
281  Taiwan GuideWithMe is an offline travel guide,...
282  「旅行台灣」是交通部觀光局提供免費在地導覽程式。導覽內容包含美食、住宿、景點、商家等資訊。並...
283                            Bus tracker for Taiwan.
284  Now you do not need a physical radio through t...
5
formed dataframe of size:( 5 , 286 )
feature_names [u'02', u'04', u'160', u'2014', u'50\u5f35\u5ba2\u88fd\u5730\u5716\u5c0e\u89bd\u53ca\u885b\u661f\u5b9a\u4f4d\u670d\u52d9', u'90', u'accord', u'action', u'ad', u'addit', u'advanc', u'adventur', u'advic', u'amend', u'android', u'app', u'apphistori', u'articl', u'aspir', u'asus', u'augment', u'automat', u'autoplay', u'avail', u'base', u'batteri', u'beaten', u'broadcast', u'browsersg', u'bus', u'cancel', u'care', u'cellular', u'channel', u'channelsi', u'channelsmost', u'check', u'chosen', u'citi', u'clean', u'click', u'cn\u5feb\u4f86\u4e0b\u8f09', u'com', u'come', u'complet', u'constant', u'consumptionb', u'contai

,02,04,160,2014,50張客製地圖導覽及衛星定位服務,90,accord,action,ad,addit,...,網羅最具臺灣味的店家及絕對不能錯過的招牌商品,臺灣個人遊,臺灣北中南東怎麼玩才最深入,請依實際道路狀況行走,讓愛自助旅行的您即便手機沒有行動上網,透過活潑筆觸,這麼多精彩的旅遊資訊,還不快點跟著,還有,陸客來台自由行蔚為風潮
0,0,0,0,0,1,0,0,0,0,0,...,1,3,1,0,1,1,1,1,2,1
1,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,3,0,1,0,0,1,1,2,1,...,0,0,0,0,0,0,0,0,0,0


 In an average, one cell of a description column generates 57 features. In this way, there are 57*n_samples features geneated after vectorization.

#### Analysis of Name Field
    suggest some of the price for higer number of sale
    w1: parameterized loudness of words in context

In [137]:
col_name = appdf.Name
print col_name[col_name.str.contains('000')]


#stem_vectorizer = StemmedCountVectorizer(min_df =1, stop_words='english')
#newfeature, fmatrix, column_vectorizer = vectorize_column(appdf, 'Name', stem_vectorizer)
#print column_vectorizer.get_feature_names()
#newfeature.head(5)


1000     5000+ Cute Love SMS Collection
1003      30000+ Funny Jokes Collection
1007     90000+ SMS Messages Collection
1008     75000+ SMS Messages Collection
1578             DuPont™ Tychem® 4000 S
2591        Learn Ukrainian 6,000 Words
3945                      Sudoku 10'000
3948                 Sudoku 10'000 Free
3953                 Sudoku 10'000 Plus
4350                     1000 Aventuras
6369     フルル大辞典 ～即引き！略語・用語・薬辞典 10,000語～
6401                 MedCalc 3000 中文精华版
7228        プリンス育成☆マジLOVE3000％ for うたプリ
7322                20000 Leagues Slots
9173             Indian Recipes 10.000+
9320      2000 Piadas Engraçadas Brasil
10048    50000 Status Quotes Collection
10251                      鬼監督の1000本ノック
11380             1000 Recipes in Hindi
12334           Desert Race Toyota 1000
13523                     Taxxi 4209000
13630                مسجات +15000 رسالة
13702                    Solitaire 1000
14624         Learn Chinese 6,000 Words
14627      Learn Chinese 10000 Mandarin


#### Analysis of 'Instalations'
    It is range values.

In [226]:
col_name = appdf.Instalations
def separate_instalation_column(dataframe, column_name,return_data_type_as=None):
    
    col_name = appdf[column_name]
    ls = col_name.str.split('-').str.get(0).str.strip(' ').str.replace(',','') #series object
    hs = col_name.str.split('-').str.get(1).str.strip(' ').str.replace(',','') #series object
    
    if return_data_type_as is float64:
        ls = ls.astype(float).fillna(0.0)
        hs = hs.astype(float).fillna(0.0)
        return ls, hs
    else:
        return ls, hs
    
ls, hs = separate_instalation_column(appdf,'Instalations', float64)
appdf.installs_ls = ls
appdf.installs_hs = hs
print appdf.installs_ls.head(5) + appdf.installs_hs.head(5)

0        150
1      15000
2       6000
3    1500000
4       6000
dtype: float64


# Issues:
            Tokenization problem
            Vectorization problem memory error
            Plot the frequency distribution of price 
            Do linear regression on price and plot Predicted_price-Desired_price Vs Predicted_price